In [27]:
import Levenshtein as lev
def leven(str1, str2):
    Distance = lev.distance(str1.lower(), str2.lower())
    Ratio = lev.ratio(str1.lower(), str2.lower())
#     print(Ratio)
    if Ratio > .95:
        return True
    else:
        return False

In [28]:
import re
def merge_awards(results):
    A = []
    B = []
    C = []
    for i in range(len(results)):
        if re.search("-",results[i][0]):
            A.append(results[i])
        else:
            B.append(results[i])
    # print(A)
    for a in A:
        str1 = a[0]
        has_merge = False
        for b in B:
            str2 = b[0]
            if leven(str1,str2):
                C.append((str1,a[1]+b[1]))
#                 print(str1,"  ",str2)
                B.remove(b)
                has_merge = True
        if not has_merge:
            C.append(a)
            
                
    for b in B:
        C.append(b)
        
    return C

In [29]:
def filter_awards(results):
    awards = []
    for result in results:
        str1 = result[0]
        has_filter = False
        if awards:
            for award in awards:
                str2 = award[0]
                if leven(str1,str2):
                    has_filter = True
                    break
            if not has_filter:
                awards.append(result)
        else:
            awards.append(result)
    return awards

In [30]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

def award_handler(text): 
    filter_words = ['django unchained',]

def str_handler(text):
    size = len(text)
    i = size - 1
    idx = size
    text = text.replace(",","-")
    
    while i >= 0:
        if text[i]<='z' and text[i]>='a':
            idx = i+1 
            break
        i -= 1
    return text[:idx].strip()       

def extract_award(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
#     filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filter_words = ['golden','globes', 'goldenglobe','globes', 'glodenglobes', 'http','2013','2015','2018','2019','2020']
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words and w not in filter_words:
#             w = w.replace('-',",")
            w = w.replace('/',",")
            filtered_sentence.append(w)
    result = ""
    start = False
#     print(filtered_sentence)
#     if re.search('best screenplay - motion picture',text):
#         print(text)
        
    for word in filtered_sentence:    
        if word == 'best':
            start = True
            result += word + " "
            continue
        
        if start and (re.match('w[io]n',word) or re.match('go(es)',word) or re.match('at',word) or re.match('went to',word) or re.match('[;\?:.!@#`]',word)):
            return str_handler(result.strip())

        if start:
            result += word+" "
    return str_handler(result.strip())
#     print(result)

In [ ]:
import json
import re
import regex
from collections import Counter
tweets = json.load(open("gg2015.json"))
reg = regex.Regex()
search_terms = ['best','w[io]n','go(es)? to','went to',':']
award_names = []
for tweet in tweets:
    text = tweet['text'].lower()
    if 'RT' not in tweet['text']:
        if re.search(search_terms[0],text) and (re.search(search_terms[1],text) or re.search(search_terms[2],text) or re.search(search_terms[3],text) or re.search(search_terms[4],text)):
            extract_str = extract_award(text)
            if re.search('usa',extract_str):
                continue
            if len(extract_str.split())<4:
                continue
#             print(extract_str)
            award_names.append(extract_str)
results = Counter(award_names).most_common()   
results = merge_awards(results)
# results = Counter(results).most_common() 
results = sorted(results, key = lambda x: x[1],reverse = True)
results = filter_awards(results)
results

In [28]:
file = json.load(open("gg2013answers.json"))
data = file['award_data']
for key in data.keys():
    print(key)

best screenplay - motion picture
best director - motion picture
best performance by an actress in a television series - comedy or musical
best foreign language film
best performance by an actor in a supporting role in a motion picture
best performance by an actress in a supporting role in a series, mini-series or motion picture made for television
best motion picture - comedy or musical
best performance by an actress in a motion picture - comedy or musical
best mini-series or motion picture made for television
best original score - motion picture
best performance by an actress in a television series - drama
best performance by an actress in a motion picture - drama
cecil b. demille award
best performance by an actor in a motion picture - comedy or musical
best motion picture - drama
best performance by an actor in a supporting role in a series, mini-series or motion picture made for television
best performance by an actress in a supporting role in a motion picture
best television serie